# Limits & Derivatives

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** 1-28
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import symbols, limit, diff, sin, cos, exp, log, oo, Rational
from sympy import series, factorial, sqrt, pi, simplify, latex

%matplotlib inline

# Publication-quality defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'lines.linewidth': 2,
    'legend.fontsize': 11,
    'figure.dpi': 100,
})

# SymPy pretty printing
sp.init_printing(use_unicode=True)
print('Libraries loaded successfully.')

## 1. The Epsilon-Delta Definition of a Limit

The formal definition of a limit is the bedrock of rigorous calculus:

$$\lim_{x \to a} f(x) = L$$

means: for every $\varepsilon > 0$, there exists $\delta > 0$ such that

$$0 < |x - a| < \delta \implies |f(x) - L| < \varepsilon.$$

**Intuition:** No matter how tight the tolerance $\varepsilon$ around $L$,
we can always find a neighborhood $\delta$ around $a$ that keeps $f(x)$
within that tolerance.

### Visual Exploration

The code below draws the $\varepsilon$-$\delta$ "box" for
$f(x) = x^2$ at $a = 2$, $L = 4$. Shrink $\varepsilon$ and watch $\delta$
contract accordingly.

In [ ]:
# Epsilon-delta visualization for f(x) = x^2 at a=2, L=4
def epsilon_delta_plot(f, a, L, epsilons):
    '''Visualize epsilon-delta definition for several epsilon values.'''
    fig, axes = plt.subplots(1, len(epsilons), figsize=(5 * len(epsilons), 5))
    if len(epsilons) == 1:
        axes = [axes]

    x = np.linspace(a - 2, a + 2, 500)

    for ax, eps in zip(axes, epsilons):
        # Find delta: we need |x^2 - 4| < eps => x in (sqrt(4-eps), sqrt(4+eps))
        # so delta = min(a - sqrt(L-eps), sqrt(L+eps) - a)
        delta = min(a - np.sqrt(max(L - eps, 0)), np.sqrt(L + eps) - a)

        ax.plot(x, f(x), 'b-', label=r'$f(x) = x^2$')
        ax.axhline(L, color='gray', ls='--', alpha=0.5)
        ax.axvline(a, color='gray', ls='--', alpha=0.5)

        # Epsilon band (horizontal)
        ax.axhspan(L - eps, L + eps, alpha=0.15, color='green',
                    label=rf'$\varepsilon = {eps}$')
        # Delta band (vertical)
        ax.axvspan(a - delta, a + delta, alpha=0.15, color='red',
                    label=rf'$\delta = {delta:.4f}$')

        ax.set_xlim(a - 2, a + 2)
        ax.set_ylim(L - 3, L + 3)
        ax.set_xlabel('x')
        ax.set_ylabel('f(x)')
        ax.set_title(rf'$\varepsilon = {eps}$')
        ax.legend(loc='upper left', fontsize=9)

    fig.suptitle(r'Epsilon-Delta: $\lim_{x\to 2} x^2 = 4$', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

epsilon_delta_plot(lambda x: x**2, a=2, L=4, epsilons=[1.0, 0.5, 0.1])
print('As epsilon shrinks, delta must shrink proportionally.')

## 2. Computing Limits with SymPy and NumPy

SymPy gives exact symbolic limits; NumPy lets us verify numerically.

### Key limit laws

| Law | Formula |
|---|---|
| Sum | $\lim (f+g) = \lim f + \lim g$ |
| Product | $\lim (fg) = (\lim f)(\lim g)$ |
| Quotient | $\lim (f/g) = \lim f / \lim g$ if $\lim g \neq 0$ |
| Squeeze | If $g \le f \le h$ and $\lim g = \lim h = L$, then $\lim f = L$ |

### Famous limits

$$\lim_{x\to 0} \frac{\sin x}{x} = 1, \qquad \lim_{x\to\infty}\left(1+\frac{1}{x}\right)^x = e$$

In [ ]:
# Symbolic limit computation with SymPy
x = symbols('x')

# Collection of important limits
limits = [
    (sin(x)/x,         x, 0,  'sin(x)/x as x->0'),
    ((1 + 1/x)**x,     x, oo, '(1 + 1/x)^x as x->inf'),
    ((exp(x) - 1)/x,   x, 0,  '(e^x - 1)/x as x->0'),
    (x * log(x),        x, 0,  'x*ln(x) as x->0+'),
    ((1 - cos(x))/x**2, x, 0,  '(1-cos x)/x^2 as x->0'),
]

print('Symbolic Limits (SymPy)')
print('=' * 55)
for expr, var, point, desc in limits:
    result = limit(expr, var, point)
    print(f'  {desc:35s} = {result}')

# Numerical verification of sin(x)/x -> 1
print('\nNumerical verification: sin(x)/x near x = 0')
for val in [1.0, 0.1, 0.01, 0.001, 1e-6]:
    print(f'  x = {val:10.6f}  =>  sin(x)/x = {np.sin(val)/val:.12f}')

## 3. Derivative Rules -- Symbolic and Numerical

The derivative of $f$ at $x = a$ is defined as

$$f'(a) = \lim_{h\to 0} \frac{f(a+h) - f(a)}{h}.$$

### Core rules

| Rule | Formula |
|---|---|
| Power | $\frac{d}{dx} x^n = n x^{n-1}$ |
| Product | $(fg)' = f'g + fg'$ |
| Quotient | $\left(\frac{f}{g}\right)' = \frac{f'g - fg'}{g^2}$ |
| Chain | $\frac{d}{dx}f(g(x)) = f'(g(x))\,g'(x)$ |

Below we verify each rule symbolically with SymPy and numerically with finite differences.

In [ ]:
# Symbolic derivatives -- verifying the four main rules
x = symbols('x')

# 1. Power rule
f_power = x**5
print('POWER RULE')
print(f'  f(x) = {f_power}')
print(f"  f'(x) = {diff(f_power, x)}   (expect 5x^4)")

# 2. Product rule
f1, f2 = sin(x), x**3
product = f1 * f2
product_deriv = diff(product, x)
manual_product = diff(f1, x)*f2 + f1*diff(f2, x)
print('\nPRODUCT RULE')
print(f'  f(x) = sin(x) * x^3')
print(f"  f'(x) = {simplify(product_deriv)}")
print(f'  Manual check matches: {simplify(product_deriv - manual_product) == 0}')

# 3. Quotient rule
numer, denom = exp(x), x**2 + 1
quotient = numer / denom
quotient_deriv = diff(quotient, x)
manual_quotient = (diff(numer, x)*denom - numer*diff(denom, x)) / denom**2
print('\nQUOTIENT RULE')
print(f'  f(x) = e^x / (x^2 + 1)')
print(f"  f'(x) = {simplify(quotient_deriv)}")
print(f'  Manual check matches: {simplify(quotient_deriv - manual_quotient) == 0}')

# 4. Chain rule
inner = x**2 + 1
outer = sp.sqrt(inner)
chain_deriv = diff(outer, x)
manual_chain = (1/(2*sp.sqrt(inner))) * diff(inner, x)
print('\nCHAIN RULE')
print(f'  f(x) = sqrt(x^2 + 1)')
print(f"  f'(x) = {simplify(chain_deriv)}")
print(f'  Manual check matches: {simplify(chain_deriv - manual_chain) == 0}')

In [ ]:
# Numerical derivative verification via finite differences
def numerical_derivative(f, x0, h=1e-8):
    '''Central difference approximation: f\'(x) = [f(x+h) - f(x-h)] / (2h).'''
    return (f(x0 + h) - f(x0 - h)) / (2 * h)

# Test functions and their known derivatives
test_cases = [
    ('x^5',            lambda x: x**5,                lambda x: 5*x**4),
    ('sin(x)*x^3',     lambda x: np.sin(x)*x**3,
     lambda x: np.cos(x)*x**3 + np.sin(x)*3*x**2),
    ('e^x/(x^2+1)',   lambda x: np.exp(x)/(x**2+1),
     lambda x: np.exp(x)*(x**2 - 2*x + 1)/(x**2+1)**2),
    ('sqrt(x^2+1)',    lambda x: np.sqrt(x**2+1),     lambda x: x/np.sqrt(x**2+1)),
]

x0 = 1.5  # Test point
print(f'Numerical vs analytical derivatives at x = {x0}')
print(f'{"Function":20s} {"Numerical":>14s} {"Analytical":>14s} {"Error":>12s}')
print('-' * 64)
for name, f, df in test_cases:
    num = numerical_derivative(f, x0)
    exact = df(x0)
    print(f'{name:20s} {num:14.10f} {exact:14.10f} {abs(num-exact):12.2e}')

## 4. Visualizing Functions, Tangent Lines, and Derivatives

A tangent line at $x = a$ has the equation

$$y = f(a) + f'(a)(x - a).$$

The plot below shows $f(x)$, its tangent at a chosen point, and the
derivative function $f'(x)$ side by side.

In [ ]:
# Function, tangent line, and derivative visualization
x_sym = symbols('x')
f_sym = sin(x_sym) * exp(-x_sym / 5)  # Damped sine wave
f_prime_sym = diff(f_sym, x_sym)

# Convert to numerical functions
f_num = sp.lambdify(x_sym, f_sym, 'numpy')
fp_num = sp.lambdify(x_sym, f_prime_sym, 'numpy')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x_vals = np.linspace(0, 15, 500)

# Left panel: function + tangent lines at several points
ax1.plot(x_vals, f_num(x_vals), 'b-', label=r'$f(x) = \sin(x)\, e^{-x/5}$')
tangent_points = [2, 5, 8, 11]
colors = ['red', 'green', 'orange', 'purple']
for a, c in zip(tangent_points, colors):
    fa = f_num(a)
    fpa = fp_num(a)
    x_tan = np.linspace(a - 1.5, a + 1.5, 50)
    y_tan = fa + fpa * (x_tan - a)
    ax1.plot(x_tan, y_tan, color=c, ls='--', lw=1.5,
             label=rf'Tangent at $x={a}$')
    ax1.plot(a, fa, 'o', color=c, markersize=6)

ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')
ax1.set_title('Function and Tangent Lines')
ax1.legend(fontsize=9, loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-1.0, 1.2)

# Right panel: derivative
ax2.plot(x_vals, fp_num(x_vals), 'r-', label=r"$f'(x)$")
ax2.axhline(0, color='gray', ls='--', alpha=0.5)
ax2.set_xlabel('x')
ax2.set_ylabel("f'(x)")
ax2.set_title('Derivative Function')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-1.0, 1.0)

plt.suptitle(r'Damped Sine Wave: $f(x) = \sin(x)\,e^{-x/5}$', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## 5. L'Hopital's Rule

When a limit yields the indeterminate forms $\frac{0}{0}$ or
$\frac{\pm\infty}{\pm\infty}$, we can apply L'Hopital's rule:

$$\lim_{x\to a}\frac{f(x)}{g(x)} = \lim_{x\to a}\frac{f'(x)}{g'(x)}$$

provided the right-hand limit exists.

### Example forms

| Type | Example |
|---|---|
| $0/0$ | $\displaystyle\lim_{x\to 0}\frac{\sin x}{x}$ |
| $\infty/\infty$ | $\displaystyle\lim_{x\to\infty}\frac{\ln x}{x}$ |
| $0 \cdot \infty$ | $\displaystyle\lim_{x\to 0^+} x\ln x$ (rewrite as $\ln x / (1/x)$) |
| $\infty - \infty$ | $\displaystyle\lim_{x\to 0}\left(\frac{1}{x} - \frac{1}{\sin x}\right)$ |

In [ ]:
# L'Hopital's Rule -- worked examples
x = symbols('x')

examples = [
    # (numerator, denominator, variable, point, description)
    (sin(x), x, x, 0, 'sin(x)/x  [0/0]'),
    (exp(x) - 1 - x, x**2, x, 0, '(e^x - 1 - x)/x^2  [0/0]'),
    (log(x), x, x, oo, 'ln(x)/x  [inf/inf]'),
    (x**3, exp(x), x, oo, 'x^3/e^x  [inf/inf]'),
    (1 - cos(x), x**2, x, 0, '(1-cos x)/x^2  [0/0]'),
]

print("L'Hopital's Rule Examples")
print('=' * 65)
for num, den, var, pt, desc in examples:
    # Direct limit (SymPy handles L'Hopital internally)
    result = limit(num / den, var, pt)

    # Show the derivative step
    num_prime = diff(num, var)
    den_prime = diff(den, var)
    step_result = limit(num_prime / den_prime, var, pt)

    print(f'\n  {desc}')
    print(f'    Direct limit:     {result}')
    print(f"    After L'Hopital:  lim {num_prime}/{den_prime} = {step_result}")
    print(f'    Match: {result == step_result}')

## 6. Taylor Series and Convergence

The Taylor series of $f(x)$ centered at $a$ is

$$f(x) = \sum_{n=0}^{\infty} \frac{f^{(n)}(a)}{n!}(x - a)^n.$$

For $a = 0$ (Maclaurin series), key expansions include:

$$e^x = \sum_{n=0}^{\infty}\frac{x^n}{n!}, \qquad
\sin x = \sum_{n=0}^{\infty}\frac{(-1)^n x^{2n+1}}{(2n+1)!}, \qquad
\cos x = \sum_{n=0}^{\infty}\frac{(-1)^n x^{2n}}{(2n)!}$$

### Convergence

The Taylor remainder theorem bounds the truncation error:

$$|R_n(x)| \le \frac{M}{(n+1)!}|x - a|^{n+1}$$

where $M = \max|f^{(n+1)}(c)|$ on the interval.

In [ ]:
# Taylor series expansion and convergence visualization
x = symbols('x')

# Functions to expand
functions = {
    r'$e^x$': exp(x),
    r'$\sin(x)$': sin(x),
    r'$\cos(x)$': cos(x),
    r'$\ln(1+x)$': log(1 + x),
}

# Print symbolic Taylor expansions
print('Taylor Series Expansions (about x = 0)')
print('=' * 60)
for name, f_sym in functions.items():
    expansion = series(f_sym, x, 0, n=8)
    print(f'\n  {name}:')
    print(f'    {expansion}')

# Convergence plot: increasing orders of Taylor polynomial
x_vals = np.linspace(-3, 3, 300)

fig, axes = plt.subplots(2, 2, figsize=(13, 10))
func_data = [
    (r'$e^x$', exp(x), np.exp),
    (r'$\sin(x)$', sin(x), np.sin),
    (r'$\cos(x)$', cos(x), np.cos),
    (r'$\ln(1+x)$', log(1+x), lambda v: np.log(1+v)),
]

orders = [1, 3, 5, 7, 11]
colors_t = plt.cm.viridis(np.linspace(0.2, 0.9, len(orders)))

for ax, (name, f_sym, f_np) in zip(axes.flat, func_data):
    y_exact = f_np(x_vals)
    # Clip for ln(1+x) domain
    if 'ln' in name:
        mask = x_vals > -0.99
        ax.plot(x_vals[mask], y_exact[mask], 'k-', lw=2.5, label='Exact')
    else:
        ax.plot(x_vals, y_exact, 'k-', lw=2.5, label='Exact')

    for order, col in zip(orders, colors_t):
        taylor_poly = sp.Poly(series(f_sym, x, 0, n=order+1).removeO(), x)
        taylor_func = sp.lambdify(x, taylor_poly.as_expr(), 'numpy')
        y_taylor = taylor_func(x_vals)

        if 'ln' in name:
            ax.plot(x_vals[mask], np.clip(y_taylor[mask], -5, 5),
                    '--', color=col, lw=1.3, label=f'Order {order}')
        else:
            ax.plot(x_vals, np.clip(y_taylor, -10, 10),
                    '--', color=col, lw=1.3, label=f'Order {order}')

    ax.set_title(f'Taylor Approx: {name}', fontsize=13)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend(fontsize=8, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-5, 5)

plt.suptitle('Taylor Series Convergence', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

## 7. Quantum Mechanics Connection

### Derivatives and Time Evolution

In quantum mechanics, the **time evolution** of a state is governed by the
Schrodinger equation:

$$i\hbar \frac{\partial}{\partial t}|\Psi(t)\rangle = \hat{H}|\Psi(t)\rangle$$

The derivative $\frac{\partial}{\partial t}$ is precisely the concept we have
been studying. The Hamiltonian $\hat{H}$ generates infinitesimal time
translations, and the solution involves the **matrix exponential** -- itself
a Taylor series:

$$|\Psi(t)\rangle = e^{-i\hat{H}t/\hbar}|\Psi(0)\rangle
= \sum_{n=0}^{\infty}\frac{1}{n!}\left(\frac{-i\hat{H}t}{\hbar}\right)^n|\Psi(0)\rangle$$

### Taylor Expansion in QM

The **translation operator** in quantum mechanics uses a Taylor expansion:

$$\hat{T}(a) = e^{-ia\hat{p}/\hbar} = \sum_{n=0}^{\infty}\frac{1}{n!}
\left(\frac{-ia\hat{p}}{\hbar}\right)^n$$

This shifts a wavefunction: $\hat{T}(a)\psi(x) = \psi(x - a)$.

In [ ]:
# QM Connection: Time evolution via Taylor expansion of the propagator
# Simulate a simple 2-level system (qubit) evolving under H = sigma_z

import numpy as np

# Pauli-Z matrix (energy eigenbasis)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

# Initial state: |+> = (|0> + |1>)/sqrt(2)
psi0 = np.array([1, 1], dtype=complex) / np.sqrt(2)

# Time evolution: exact vs Taylor approximation
hbar = 1.0  # Natural units
H = sigma_z

t_values = np.linspace(0, 2*np.pi, 200)
exact_prob0 = []       # |<0|psi(t)>|^2  (exact)
taylor_prob0 = {}      # Taylor approximations at various orders

# Exact evolution
for t in t_values:
    # U(t) = exp(-iHt/hbar) -- for diagonal H this is simple
    U_exact = np.diag([np.exp(-1j*t), np.exp(1j*t)])
    psi_t = U_exact @ psi0
    exact_prob0.append(np.abs(psi_t[0])**2)

# Taylor-approximated evolution
for order in [1, 3, 5, 10]:
    probs = []
    for t in t_values:
        # U_taylor = sum_{n=0}^{order} (-iHt)^n / n!
        U_taylor = np.eye(2, dtype=complex)
        term = np.eye(2, dtype=complex)
        for n in range(1, order + 1):
            term = term @ (-1j * H * t) / n
            U_taylor += term
        psi_t = U_taylor @ psi0
        probs.append(np.abs(psi_t[0])**2)
    taylor_prob0[order] = probs

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_values, exact_prob0, 'k-', lw=2.5, label='Exact $e^{-iHt}$')
for order, probs in taylor_prob0.items():
    ax.plot(t_values, probs, '--', lw=1.5, label=f'Taylor order {order}')

ax.set_xlabel('Time $t$')
ax.set_ylabel(r'$|\langle 0|\Psi(t)\rangle|^2$')
ax.set_title('Qubit Time Evolution: Exact vs Taylor Expansion of Propagator')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.1, 1.4)
plt.tight_layout()
plt.show()
print('Higher Taylor orders converge to the exact oscillation.')

## 8. Higher-Order Derivatives and Curvature

The **second derivative** $f''(x)$ measures the rate of change of the slope,
giving us information about **concavity**:

- $f''(x) > 0$: concave up (curve bends upward)
- $f''(x) < 0$: concave down (curve bends downward)
- $f''(x) = 0$: potential inflection point

The **curvature** of a plane curve is

$$\kappa(x) = \frac{|f''(x)|}{(1 + [f'(x)]^2)^{3/2}}$$

which quantifies how sharply the curve bends at each point.

In [ ]:
# Higher-order derivatives and curvature
x_sym = symbols('x')
f_sym = x_sym**3 - 3*x_sym**2 + 2  # Cubic polynomial

f1 = diff(f_sym, x_sym)         # First derivative
f2 = diff(f_sym, x_sym, 2)      # Second derivative
f3 = diff(f_sym, x_sym, 3)      # Third derivative

print('f(x)   =', f_sym)
print("f'(x)  =", f1)
print("f''(x) =", f2)
print("f'''(x)=", f3)

# Convert to numerical
f_num = sp.lambdify(x_sym, f_sym, 'numpy')
f1_num = sp.lambdify(x_sym, f1, 'numpy')
f2_num = sp.lambdify(x_sym, f2, 'numpy')

x_vals = np.linspace(-1, 4, 400)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# f(x)
ax1.plot(x_vals, f_num(x_vals), 'b-', lw=2)
ax1.axhline(0, color='gray', ls='--', alpha=0.4)
ax1.set_ylabel('f(x)')
ax1.set_title(r'$f(x) = x^3 - 3x^2 + 2$ and its derivatives')
ax1.grid(True, alpha=0.3)

# f'(x) -- mark critical points
ax2.plot(x_vals, f1_num(x_vals), 'r-', lw=2)
ax2.axhline(0, color='gray', ls='--', alpha=0.4)
# Critical points: f'(x) = 3x^2 - 6x = 3x(x-2) = 0 => x=0, x=2
for cp in [0, 2]:
    ax2.plot(cp, 0, 'ko', markersize=8)
    ax2.annotate(f'x={cp}', (cp, 0), textcoords='offset points',
                 xytext=(10, 10), fontsize=11)
ax2.set_ylabel("f'(x)")
ax2.grid(True, alpha=0.3)

# f''(x) -- mark inflection point
ax3.plot(x_vals, f2_num(x_vals), 'g-', lw=2)
ax3.axhline(0, color='gray', ls='--', alpha=0.4)
# Inflection: f''(x) = 6x - 6 = 0 => x=1
ax3.plot(1, 0, 'ko', markersize=8)
ax3.annotate('Inflection x=1', (1, 0), textcoords='offset points',
             xytext=(10, 10), fontsize=11)
# Shade concavity regions
ax3.fill_between(x_vals, f2_num(x_vals), 0,
                 where=f2_num(x_vals) > 0, alpha=0.15, color='blue',
                 label='Concave up')
ax3.fill_between(x_vals, f2_num(x_vals), 0,
                 where=f2_num(x_vals) < 0, alpha=0.15, color='red',
                 label='Concave down')
ax3.set_xlabel('x')
ax3.set_ylabel("f''(x)")
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

### Key Formulas

| Concept | Formula |
|---|---|
| Limit definition | $\forall\varepsilon>0,\;\exists\delta>0:\; 0<|x-a|<\delta \Rightarrow |f(x)-L|<\varepsilon$ |
| Derivative | $f'(a) = \lim_{h\to 0}\frac{f(a+h)-f(a)}{h}$ |
| Product rule | $(fg)' = f'g + fg'$ |
| Chain rule | $(f\circ g)' = f'(g)\cdot g'$ |
| L'Hopital | $\lim\frac{f}{g} = \lim\frac{f'}{g'}$ (for $0/0$ or $\infty/\infty$) |
| Taylor series | $f(x) = \sum_{n=0}^{\infty}\frac{f^{(n)}(a)}{n!}(x-a)^n$ |
| Curvature | $\kappa = \frac{|f''|}{(1+f'^2)^{3/2}}$ |

### Main Takeaways

1. The epsilon-delta definition makes limits rigorous and is the foundation for all of calculus.
2. Derivative rules (power, product, quotient, chain) can all be verified symbolically with SymPy and numerically with finite differences.
3. L'Hopital's rule resolves indeterminate forms by differentiating numerator and denominator.
4. Taylor series approximate functions as polynomials -- the same idea appears in quantum mechanics for the time-evolution operator.
5. The Schrodinger equation is fundamentally a differential equation; everything in this month connects directly to quantum physics.

---
*Next month: Integration and Infinite Series (Days 29-56)*